In [1]:
from langchain_chroma import Chroma
import os
from pyprojroot import here
from langchain_community.embeddings import HuggingFaceEmbeddings
from groq import Groq
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

**Load environment variables and configs**

In [2]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

**Load the vectorDB**

In [3]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

C:\Users\Arushi Saxena\AppData\Local\Temp\ipykernel_2532\1379456849.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
c:\Users\Arushi Saxena\anaconda3\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of vectors in vectordb: 22 




**Sample Query**

In [4]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

**Perform the vector Search**

In [5]:
docs = vectordb.similarity_search(message, k=K)

In [6]:
docs

[Document(id='dc87dfaf-4fa4-4f72-9f08-85b5dc56ee79', metadata={'creationdate': '2024-09-08T21:34:59+00:00', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'source': 'C:\\data_analyst_bot\\data\\unstructured_docs\\swiss_airline_policy\\swiss_faq.pdf', 'total_pages': 12, 'producer': 'PDFCreator Online (www.pdfforge.org/online)', 'title': 'Merged with PDFCreator Online', 'page_label': '10', 'page': 9, 'moddate': '2024-09-08T21:34:59+00:00'}, page_content="for a refund or may only be able to receive a partial refund. If you booked your flight through a third-party website or\ntravel agent, you may need to contact them directly to cancel your flight. Always check the terms and conditions of your\nticket to make sure you understand the cancellation policy and any associated fees or penalties. If you're cancelling your\nflight due to unforeseen circumstances such as a medical emergency or a natural disaster, Swiss Air may offer you\nspecial exemptions or accommodations. What is Swi

**Prepare the prompt for the GPT model**

In [7]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

Prepared prompt

In [8]:
pprint(prompt)

('# Content:\n'
 'for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster, Swiss Air may offer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'How to Cancel a Swiss Air Flight: 877-\n'
 '5O7-7341 Step-by-Step Guide\n'
 'Swiss Air is a premium airline based in Switzerland that offers a range of '
 'domestic and international flights to\n'
 'passengers. However, sometimes situations arise where passengers may need to '
 'cancel their flights. In such cases, it is\n'
 'important 

**Pass the prompt to the GPT model and get the response**

In [9]:
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."},
        {"role": "user", "content": prompt}
    ]
)

Printing the response

In [10]:
pprint(response.choices[0].message.content)

('According to the content, Swiss Airlines has a 24-hour window to cancel a '
 'flight without penalty for fare types like Flex and Business Flex. For other '
 'fare types, cancellation fees may apply, and the fee amount depends on the '
 'fare type and the time of cancellation.')


**RAG Tool design using LangChain**

In [13]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [14]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [15]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster, Swiss Air may offer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'hour cancellation policy that allows passengers to cancel their flights '
 'within 24 hours of booking at +1-877-507-7341\n'
 'without penalty. This policy applies to all fare types, including '
 'non-refundable tickets. If you cancel your Swiss Airlines\n'
 "flight within 24 hours of booking, you'll receive a full refund of your "
 'ticket price.\n'
 